#setup
- connect to google drive in this step

In [ ]:
import pandas as pd
import math
import numpy as np
import os
import glob
import sklearn
import pickle
import glob
import scipy.io
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
project_path = '/content/drive/MyDrive/demo-me-2021-07-14'
output_dir = os.path.join(project_path, "behaviors/")
if not os.path.exists(output_dir):
  os.mkdir(output_dir)

In [ ]:
def get_data(individual, bodypart, h5_file):
  mouse_data = h5_file.xs(individual,level='individuals',axis=1)
  out_data = mouse_data.xs(bodypart,level='bodyparts',axis=1)
  out_data.columns = out_data.columns.droplevel("scorer")
  output = out_data.copy()
  return output

In [ ]:
def within_area(area_vector, input_coor):
  area_startx = area_vector[0]
  area_starty = area_vector[1]
  area_distx = area_vector[2]
  area_disty = area_vector[3]
  x = input_coor["x"].iloc[0]
  y = input_coor["y"].iloc[0]
  if (area_startx <= x <= (area_startx+area_distx)) and (area_starty <= y <= (area_starty+area_disty)):
    result = 1
  else:
    result = 0
  return result

In [ ]:
def euclid_dist(point1_coor, point2_coor):
  point1 = np.array((point1_coor["x"].iloc[0], point1_coor["y"].iloc[0]))
  point2 = np.array((point2_coor["x"].iloc[0], point2_coor["y"].iloc[0]))
  output_dist = np.linalg.norm(point1 - point2)
  return output_dist

In [ ]:
def change_behavior(beh_data, frame_range_start, frame_range_end, new_behavior):
  corrected_beh_data = beh_data.copy()
  for frame_i in range(frame_range_start, frame_range_end + 1):
    corrected_beh_data["behavior"][frame_i] = new_behavior
  return corrected_beh_data

#read previous behavior data pickle

In [ ]:
video_name = 'PZ71_1'

behavior_file = open(output_dir + video_name + '_behavior.pickle', 'rb')
behavior = pickle.load(behavior_file)
behavior_file.close()

behavior_copy = behavior.truncate(after=(36001 - 1)).copy()

#correct corrected_behavior

In [ ]:
start_i = 19900
end_i = 19986
new_behavior = 'mount'
corrected_behavior = change_behavior(behavior_copy, start_i, end_i, new_behavior)

In [ ]:
with open(output_dir + video_name + '_corrected_behavior.pickle', 'wb') as f:
    pickle.dump(corrected_behavior, f)

In [ ]:
print(behavior_copy["behavior"][start_i])
print(corrected_behavior["behavior"][start_i])

nan
mount


#make new mount_events

In [ ]:
mount_events = pd.DataFrame(index=range(36001 - 1), columns=['start_index', 'minutes', 'seconds', 'end_index'])
events_n = 0
previous_event_was_mount = 0
for i in range(1, 36001):
  if corrected_behavior["behavior"][i] != corrected_behavior["behavior"][i-1]:
    if corrected_behavior["behavior"][i] == 'mount':
      mount_events["start_index"][events_n] = i
      mount_events["minutes"][events_n] = i / (30*60)
      mount_events["seconds"][events_n] = round(i / 30) % 60
      events_n = events_n + 1
      previous_event_was_mount = 1
    elif previous_event_was_mount == 1:
      mount_events["end_index"][events_n - 1] = i
      previous_event_was_mount = 0
    else:
      previous_event_was_mount = 0

if corrected_behavior["behavior"][36000] == 'mount':
  mount_events["end_index"][events_n - 1] = 36000

print(events_n)
mount_events = mount_events.dropna(how='all')

61


In [ ]:
print(mount_events)

   start_index    minutes seconds end_index
0          304   0.168889      10       319
1          634   0.352222      21       690
2         4357   2.420556      25      4448
3         4471   2.483889      29      4611
4         4612   2.562222      34      4636
..         ...        ...     ...       ...
56       34943  19.412778      25     34968
57       34987  19.437222      26     35015
58       35016  19.453333      27     35055
59       35069  19.482778      29     35089
60       35090  19.494444      30     35184

[61 rows x 4 columns]


In [ ]:
with open(output_dir + video_name + '_mount_events.pickle', 'wb') as f:
    pickle.dump(mount_events, f)